<a href="https://colab.research.google.com/github/rromerov/Proyecto_Integrador/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análisis exploratorio de los datos

## Instalar librerias faltantes

In [1]:
! pip install chembl_webresource_client
! pip install --extra-index-url=https://pypi.nvidia.com cudf-cu12

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.2/55.2 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.3/473.3 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.6/582.6 kB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.3/30.3 MB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 80.5 MB/s eta 0:00:00


## Importar librerias

In [77]:
%load_ext cudf.pandas
import pandas as pd
from chembl_webresource_client.new_client import new_client
from google.colab import drive
import locale

The cudf.pandas extension is already loaded. To reload it, use:
  %reload_ext cudf.pandas


## Buscar proteina target, en este caso VEGF165

In [17]:
# Búsqueda de VEGF165 target
target = new_client.target
target_query = target.search('Neuropilin-1')
targets = pd.DataFrame.from_dict(target_query)
targets

,cross_references,organism,pref_name,score,species_group_flag,target_chembl_id,target_components,target_type,tax_id
0,"[{'xref_id': 'O14786', 'xref_name': None, 'xre...",Homo sapiens,Neuropilin-1,21.0,False,CHEMBL5174,"[{'accession': 'O14786', 'component_descriptio...",SINGLE PROTEIN,9606.0
1,[],Rattus norvegicus,Neuropilin-1,21.0,False,CHEMBL3309098,"[{'accession': 'Q9QWJ9', 'component_descriptio...",SINGLE PROTEIN,10116.0
2,[],Homo sapiens,Neuropilin-2,17.0,False,CHEMBL4295667,"[{'accession': 'O60462', 'component_descriptio...",SINGLE PROTEIN,9606.0
3,"[{'xref_id': 'P30411', 'xref_name': None, 'xre...",Homo sapiens,Bradykinin B2 receptor,8.0,False,CHEMBL3157,"[{'accession': 'P30411', 'component_descriptio...",SINGLE PROTEIN,9606.0
4,"[{'xref_id': 'P13497', 'xref_name': None, 'xre...",Homo sapiens,Bone morphogenetic protein 1,8.0,False,CHEMBL3898,"[{'accession': 'P13497', 'component_descriptio...",SINGLE PROTEIN,9606.0
...,...,...,...,...,...,...,...,...,...
3898,[],Mus musculus,Inhibitor of nuclear factor kappa-B kinase sub...,0.0,False,CHEMBL4296092,"[{'accession': 'Q60680', 'component_descriptio...",PROTEIN COMPLEX,10090.0
3899,[],Zika virus,Genome polyprotein,0.0,False,CHEMBL4523307,"[{'accession': 'Q32ZE1', 'component_descriptio...",SINGLE PROTEIN,64320.0
3900,[],Severe acute respiratory syndrome coronavirus 2,Replicase polyprotein 1ab,0.0,False,CHEMBL4523582,"[{'accession': 'P0DTD1', 'component_descriptio...",SINGLE PROTEIN,2697049.0
3901,[],Yellow fever virus (strain 17D vaccine) (YFV),Genome polyprotein,0.0,False,CHEMBL4523585,"[{'accession': 'P03314', 'component_descriptio...",SINGLE PROTEIN,11090.0


## Selecionar y recuperar los datos de bioactividad para la proteina VEGF165

In [48]:
selected_target = 'CHEMBL5174'

In [49]:
# Filtrar el dataset para solo tener la fila donde el target_chembl_id sea igual a CHEMBL5174
selected_target_row = targets[targets['target_chembl_id'] == selected_target]
selected_target_row

,cross_references,organism,pref_name,score,species_group_flag,target_chembl_id,target_components,target_type,tax_id
0,"[{'xref_id': 'O14786', 'xref_name': None, 'xre...",Homo sapiens,Neuropilin-1,21.0,False,CHEMBL5174,"[{'accession': 'O14786', 'component_descriptio...",SINGLE PROTEIN,9606.0


Aqui vamos a recuperar los datos de bioactividad para la proteina VEGF165

In [60]:
activity = new_client.activity
res = activity.filter(target_chembl_id=selected_target).filter(standard_type='IC50')

In [64]:
pd.set_option('display.max_columns', None)
# Guardar response como dataframe
df = pd.DataFrame.from_dict(res)
df.head()

,action_type,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,bao_label,canonical_smiles,data_validity_comment,data_validity_description,document_chembl_id,document_journal,document_year,ligand_efficiency,molecule_chembl_id,molecule_pref_name,parent_molecule_chembl_id,pchembl_value,potential_duplicate,qudt_units,record_id,relation,src_id,standard_flag,standard_relation,standard_text_value,standard_type,standard_units,standard_upper_value,standard_value,target_chembl_id,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,None,1747710,[],CHEMBL867991,Inhibition of VEGF165-NRP1 interaction by ELISA,B,None,None,BAO_0000190,BAO_0000357,single protein format,CN(C)CCCOc1cccc2c(C#N)c(-c3ccccc3)c(NC3CCCCC3)n12,None,None,CHEMBL1146846,Bioorg Med Chem Lett,2006,"{'bei': '9.85', 'le': '0.18', 'lle': '-1.45', ...",CHEMBL212560,None,CHEMBL212560,4.10,0,http://www.openphacts.org/units/Nanomolar,548320,=,1,1,=,None,IC50,nM,None,79000.0,CHEMBL5174,Homo sapiens,Neuropilin-1,9606,None,None,IC50,uM,UO_0000065,None,79.0
1,None,None,1747711,[],CHEMBL867991,Inhibition of VEGF165-NRP1 interaction by ELISA,B,None,None,BAO_0000190,BAO_0000357,single protein format,CN(C)CCNc1cccc2c(C#N)c(-c3ccccc3)c(NC3CCCCC3)n12,None,None,CHEMBL1146846,Bioorg Med Chem Lett,2006,"{'bei': '10.90', 'le': '0.20', 'lle': '-0.82',...",CHEMBL386641,None,CHEMBL386641,4.38,0,http://www.openphacts.org/units/Nanomolar,548321,=,1,1,=,None,IC50,nM,None,42000.0,CHEMBL5174,Homo sapiens,Neuropilin-1,9606,None,None,IC50,uM,UO_0000065,None,42.0
2,None,None,1747712,[],CHEMBL867991,Inhibition of VEGF165-NRP1 interaction by ELISA,B,None,None,BAO_0000190,BAO_0000357,single protein format,N#Cc1c(-c2ccccc2)c(Nc2cccc(N)c2)n2c(Cl)cccc12,None,None,CHEMBL1146846,Bioorg Med Chem Lett,2006,"{'bei': '11.42', 'le': '0.22', 'lle': '-1.36',...",CHEMBL425440,None,CHEMBL425440,4.10,0,http://www.openphacts.org/units/Nanomolar,548327,=,1,1,=,None,IC50,nM,None,80000.0,CHEMBL5174,Homo sapiens,Neuropilin-1,9606,None,None,IC50,uM,UO_0000065,None,80.0
3,None,None,1747713,[],CHEMBL867991,Inhibition of VEGF165-NRP1 interaction by ELISA,B,None,None,BAO_0000190,BAO_0000357,single protein format,CN(C)CCCNc1c(-c2ccccc2)c(C#N)c2cccc(Cl)n12,None,None,CHEMBL1146846,Bioorg Med Chem Lett,2006,"{'bei': '11.92', 'le': '0.23', 'lle': '-0.28',...",CHEMBL436932,None,CHEMBL436932,4.21,0,http://www.openphacts.org/units/Nanomolar,548328,=,1,1,=,None,IC50,nM,None,62000.0,CHEMBL5174,Homo sapiens,Neuropilin-1,9606,None,None,IC50,uM,UO_0000065,None,62.0
4,None,None,1747714,[],CHEMBL867991,Inhibition of VEGF165-NRP1 interaction by ELISA,B,None,None,BAO_0000190,BAO_0000357,single protein format,Cc1cccc(C)c1Nc1c(-c2ccccc2)c(C#N)c2cccc(Cl)n12,None,None,CHEMBL1146846,Bioorg Med Chem Lett,2006,"{'bei': '11.35', 'le': '0.21', 'lle': '-2.27',...",CHEMBL213321,None,CHEMBL213321,4.22,0,http://www.openphacts.org/units/Nanomolar,548329,=,1,1,=,None,IC50,nM,None,60000.0,CHEMBL5174,Homo sapiens,Neuropilin-1,9606,None,None,IC50,uM,UO_0000065,None,60.0


In [91]:
records_df = len(df)
print(f'Number of records in the dataset: {records_df}')

Number of records in the dataset: 139


## Descripción de columnas relevantes dentro del dataframe:

* Standard Value: Es la potencia de la droga, mientras menor sea el valor más eficaz es, debido a que un mayor significa que se requiere de una mayor cantidad de droga para tener el mismo efecto

In [65]:
# Verificar que solo se cuente con registros de IC50
df.standard_type.unique()

array(['IC50'], dtype=object)

In [72]:
# Visualizar rango de valores de standard value
df.standard_value.unique()

array(['79000.0', '42000.0', '80000.0', '62000.0', '60000.0', '47000.0',
       '9000.0', '4000.0', '3000.0', '2000.0', '95000.0', '22000.0',
       '14000.0', '13000.0', '10000.0', '6000.0', '5000.0', '29000.0',
       '43000.0', '19000.0', '18000.0', '30000.0', '17000.0', '52000.0',
       '33000.0', '8000.0', '12000.0', '23000.0', '7000.0', '171000.0',
       '92000.0', '9300.0', '609.0', '1600.0', '800.0', '4600.0',
       '25118.86', '5400.0', '2900.0', '26000.0', '2951.21', '7762.47',
       '7700.0', '5370.32', None, '120000.0', '187000.0', '181000.0',
       '88000.0', '39000.0', '134000.0', '69000.0', '25000.0', '20900.0',
       '21300.0', '48600.0', '7200.0', '16000.0', '100000.0', '8900.0',
       '9200.0', '11700.0', '4300.0', '2300.0', '80500.0', '23400.0',
       '16100.0', '18100.0', '50100.0', '84100.0', '831763.77',
       '20892.96', '8912.51', '23442.29', '21379.62', '48977.88',
       '14125.38', '7244.36', '15848.93', '9332.54', '10715.19',
       '2884.03', '2238

# Guardar resultados en Google Drive

In [75]:
# Cargar Google Drive al notebook
drive.mount('/content/drive')

Mounted at /content/drive


In [78]:
locale.getpreferredencoding = lambda: "UTF-8"
# Crear carpeta dentro de google drive llamada data
! mkdir '/content/drive/My Drive/Colab Notebooks/data'

In [79]:
# Subir el csv a la carpeta destino
df.to_csv('/content/drive/My Drive/Colab Notebooks/data/bioactivity_data.csv', index=False)

In [81]:
# Verificar que el archivo se encuentra en la carpeta destino
! ls -l '/content/drive/My Drive/Colab Notebooks/data'

total 73
-rw------- 1 root root 74689 May  4 14:20 bioactivity_data.csv


Ver contenido de **bioactivity_data.csv**

In [85]:
! head '/content/drive/My Drive/Colab Notebooks/data/bioactivity_data.csv'

action_type,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,bao_label,canonical_smiles,data_validity_comment,data_validity_description,document_chembl_id,document_journal,document_year,ligand_efficiency,molecule_chembl_id,molecule_pref_name,parent_molecule_chembl_id,pchembl_value,potential_duplicate,qudt_units,record_id,relation,src_id,standard_flag,standard_relation,standard_text_value,standard_type,standard_units,standard_upper_value,standard_value,target_chembl_id,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
,,1747710,[],CHEMBL867991,Inhibition of VEGF165-NRP1 interaction by ELISA,B,,,BAO_0000190,BAO_0000357,single protein format,CN(C)CCCOc1cccc2c(C#N)c(-c3ccccc3)c(NC3CCCCC3)n12,,,CHEMBL1146846,Bioorg Med Chem Lett,2006,"{'bei': '9.85', 'le': '0.18', 'lle': '-1.45', 'sei': '7.78'}",CHEMBL212560,,CHEMBL212560,4.10

## Manejo de valores faltantes
Dado que nos interesa conocer el standard value, los registros que no cuenten con esta información serán eliminados

In [87]:
df2 = df[df.standard_value.notna()]
df2

,action_type,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,bao_label,canonical_smiles,data_validity_comment,data_validity_description,document_chembl_id,document_journal,document_year,ligand_efficiency,molecule_chembl_id,molecule_pref_name,parent_molecule_chembl_id,pchembl_value,potential_duplicate,qudt_units,record_id,relation,src_id,standard_flag,standard_relation,standard_text_value,standard_type,standard_units,standard_upper_value,standard_value,target_chembl_id,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,None,1747710,[],CHEMBL867991,Inhibition of VEGF165-NRP1 interaction by ELISA,B,None,None,BAO_0000190,BAO_0000357,single protein format,CN(C)CCCOc1cccc2c(C#N)c(-c3ccccc3)c(NC3CCCCC3)n12,None,None,CHEMBL1146846,Bioorg Med Chem Lett,2006,"{'bei': '9.85', 'le': '0.18', 'lle': '-1.45', ...",CHEMBL212560,None,CHEMBL212560,4.10,0,http://www.openphacts.org/units/Nanomolar,548320,=,1,1,=,None,IC50,nM,None,79000.0,CHEMBL5174,Homo sapiens,Neuropilin-1,9606,None,None,IC50,uM,UO_0000065,None,79.0
1,None,None,1747711,[],CHEMBL867991,Inhibition of VEGF165-NRP1 interaction by ELISA,B,None,None,BAO_0000190,BAO_0000357,single protein format,CN(C)CCNc1cccc2c(C#N)c(-c3ccccc3)c(NC3CCCCC3)n12,None,None,CHEMBL1146846,Bioorg Med Chem Lett,2006,"{'bei': '10.90', 'le': '0.20', 'lle': '-0.82',...",CHEMBL386641,None,CHEMBL386641,4.38,0,http://www.openphacts.org/units/Nanomolar,548321,=,1,1,=,None,IC50,nM,None,42000.0,CHEMBL5174,Homo sapiens,Neuropilin-1,9606,None,None,IC50,uM,UO_0000065,None,42.0
2,None,None,1747712,[],CHEMBL867991,Inhibition of VEGF165-NRP1 interaction by ELISA,B,None,None,BAO_0000190,BAO_0000357,single protein format,N#Cc1c(-c2ccccc2)c(Nc2cccc(N)c2)n2c(Cl)cccc12,None,None,CHEMBL1146846,Bioorg Med Chem Lett,2006,"{'bei': '11.42', 'le': '0.22', 'lle': '-1.36',...",CHEMBL425440,None,CHEMBL425440,4.10,0,http://www.openphacts.org/units/Nanomolar,548327,=,1,1,=,None,IC50,nM,None,80000.0,CHEMBL5174,Homo sapiens,Neuropilin-1,9606,None,None,IC50,uM,UO_0000065,None,80.0
3,None,None,1747713,[],CHEMBL867991,Inhibition of VEGF165-NRP1 interaction by ELISA,B,None,None,BAO_0000190,BAO_0000357,single protein format,CN(C)CCCNc1c(-c2ccccc2)c(C#N)c2cccc(Cl)n12,None,None,CHEMBL1146846,Bioorg Med Chem Lett,2006,"{'bei': '11.92', 'le': '0.23', 'lle': '-0.28',...",CHEMBL436932,None,CHEMBL436932,4.21,0,http://www.openphacts.org/units/Nanomolar,548328,=,1,1,=,None,IC50,nM,None,62000.0,CHEMBL5174,Homo sapiens,Neuropilin-1,9606,None,None,IC50,uM,UO_0000065,None,62.0
4,None,None,1747714,[],CHEMBL867991,Inhibition of VEGF165-NRP1 interaction by ELISA,B,None,None,BAO_0000190,BAO_0000357,single protein format,Cc1cccc(C)c1Nc1c(-c2ccccc2)c(C#N)c2cccc(Cl)n12,None,None,CHEMBL1146846,Bioorg Med Chem Lett,2006,"{'bei': '11.35', 'le': '0.21', 'lle': '-2.27',...",CHEMBL213321,None,CHEMBL213321,4.22,0,http://www.openphacts.org/units/Nanomolar,548329,=,1,1,=,None,IC50,nM,None,60000.0,CHEMBL5174,Homo sapiens,Neuropilin-1,9606,None,None,IC50,uM,UO_0000065,None,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,None,None,20628397,[],CHEMBL4610306,Inhibition of recombinant human biotinylated V...,B,None,None,BAO_0000190,BAO_0000219,cell-based format,N=C(N)NCCC[C@H](NC(=O)c1sccc1NS(=O)(=O)c1cccc2...,None,None,CHEMBL4609981,Bioorg Med Chem,2020,"{'bei': '9.96', 'le': '0.21', 'lle': '3.95', '...",CHEMBL1078974,None,CHEMBL1078974,4.96,0,http://www.openphacts.org/units/Nanomolar,3471124,=,1,1,=,None,IC50,nM,None,11100.0,CHEMBL5174,Homo sapiens,Neuropilin-1,9606,None,None,IC50,uM,UO_0000065,12.7,11.1
135,None,None,20628398,[],CHEMBL4610306,Inhibition of recombinant human biotinylated V...,B,None,None,BAO_0000190,BAO_0000219,cell-based format,CCc1nnc(NC(=O)CS

In [120]:
records_df2 = len(df2)
preserved_info = round((records_df2/records_df)*100,2)
print(f'Number of records in df2 {records_df2}, percentage of information preserved: {preserved_info}')

Number of records in df2 131, percentage of information preserved: 94.24


In [128]:
# Verificar la cantidad de registros de canonical smiles
canon_smiles_count = sum(df2.canonical_smiles.notnull())
canon_smiles_count

131

In [131]:
# Verificar que la columna canonical_smiles cuente con valores unicos
unique_canon_smiles_count = len(df2.canonical_smiles.unique())
percentage_unique_canon_smiles = round(unique_canon_smiles_count/canon_smiles_count*100,2)
print(f'Number of unique canonical_smiles records in df2: {unique_canon_smiles_count}, percentage of unique records: {percentage_unique_canon_smiles}')

Number of unique canonical_smiles records in df2: 98, percentage of unique records: 74.81


### Eliminar canonical_smiles duplicadas

In [132]:
df2_unique_can_smiles = df2.drop_duplicates(['canonical_smiles'])
df2_unique_can_smiles

,action_type,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,bao_label,canonical_smiles,data_validity_comment,data_validity_description,document_chembl_id,document_journal,document_year,ligand_efficiency,molecule_chembl_id,molecule_pref_name,parent_molecule_chembl_id,pchembl_value,potential_duplicate,qudt_units,record_id,relation,src_id,standard_flag,standard_relation,standard_text_value,standard_type,standard_units,standard_upper_value,standard_value,target_chembl_id,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,None,1747710,[],CHEMBL867991,Inhibition of VEGF165-NRP1 interaction by ELISA,B,None,None,BAO_0000190,BAO_0000357,single protein format,CN(C)CCCOc1cccc2c(C#N)c(-c3ccccc3)c(NC3CCCCC3)n12,None,None,CHEMBL1146846,Bioorg Med Chem Lett,2006,"{'bei': '9.85', 'le': '0.18', 'lle': '-1.45', ...",CHEMBL212560,None,CHEMBL212560,4.10,0,http://www.openphacts.org/units/Nanomolar,548320,=,1,1,=,None,IC50,nM,None,79000.0,CHEMBL5174,Homo sapiens,Neuropilin-1,9606,None,None,IC50,uM,UO_0000065,None,79.0
1,None,None,1747711,[],CHEMBL867991,Inhibition of VEGF165-NRP1 interaction by ELISA,B,None,None,BAO_0000190,BAO_0000357,single protein format,CN(C)CCNc1cccc2c(C#N)c(-c3ccccc3)c(NC3CCCCC3)n12,None,None,CHEMBL1146846,Bioorg Med Chem Lett,2006,"{'bei': '10.90', 'le': '0.20', 'lle': '-0.82',...",CHEMBL386641,None,CHEMBL386641,4.38,0,http://www.openphacts.org/units/Nanomolar,548321,=,1,1,=,None,IC50,nM,None,42000.0,CHEMBL5174,Homo sapiens,Neuropilin-1,9606,None,None,IC50,uM,UO_0000065,None,42.0
2,None,None,1747712,[],CHEMBL867991,Inhibition of VEGF165-NRP1 interaction by ELISA,B,None,None,BAO_0000190,BAO_0000357,single protein format,N#Cc1c(-c2ccccc2)c(Nc2cccc(N)c2)n2c(Cl)cccc12,None,None,CHEMBL1146846,Bioorg Med Chem Lett,2006,"{'bei': '11.42', 'le': '0.22', 'lle': '-1.36',...",CHEMBL425440,None,CHEMBL425440,4.10,0,http://www.openphacts.org/units/Nanomolar,548327,=,1,1,=,None,IC50,nM,None,80000.0,CHEMBL5174,Homo sapiens,Neuropilin-1,9606,None,None,IC50,uM,UO_0000065,None,80.0
3,None,None,1747713,[],CHEMBL867991,Inhibition of VEGF165-NRP1 interaction by ELISA,B,None,None,BAO_0000190,BAO_0000357,single protein format,CN(C)CCCNc1c(-c2ccccc2)c(C#N)c2cccc(Cl)n12,None,None,CHEMBL1146846,Bioorg Med Chem Lett,2006,"{'bei': '11.92', 'le': '0.23', 'lle': '-0.28',...",CHEMBL436932,None,CHEMBL436932,4.21,0,http://www.openphacts.org/units/Nanomolar,548328,=,1,1,=,None,IC50,nM,None,62000.0,CHEMBL5174,Homo sapiens,Neuropilin-1,9606,None,None,IC50,uM,UO_0000065,None,62.0
4,None,None,1747714,[],CHEMBL867991,Inhibition of VEGF165-NRP1 interaction by ELISA,B,None,None,BAO_0000190,BAO_0000357,single protein format,Cc1cccc(C)c1Nc1c(-c2ccccc2)c(C#N)c2cccc(Cl)n12,None,None,CHEMBL1146846,Bioorg Med Chem Lett,2006,"{'bei': '11.35', 'le': '0.21', 'lle': '-2.27',...",CHEMBL213321,None,CHEMBL213321,4.22,0,http://www.openphacts.org/units/Nanomolar,548329,=,1,1,=,None,IC50,nM,None,60000.0,CHEMBL5174,Homo sapiens,Neuropilin-1,9606,None,None,IC50,uM,UO_0000065,None,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,None,None,19481290,[],CHEMBL4480618,Inhibition of human biotinylated VEGF-A165 bin...,B,None,None,BAO_0000190,BAO_0000357,single protein format,C[C@H](NC(=O)[C@@H]1Cc2ccccc2CN1C(=O)[C@@H](N)...,None,None,CHEMBL4480373,Medchemcomm,2019,"{'bei': '6.24', 'le': '0.12', 'lle': '6.82', '...",CHEMBL4515173,None,CHEMBL4515173,4.30,0,http://www.openphacts.org/units/Nanomolar,3257738,=,1,1,=,None,IC50,nM,None,50100.0,CHEMBL5174,Homo sapiens,Neuropilin-1,9606,None,None,IC50,uM,UO_0000065,None,50.1
135,None,None,20628398,[],CHEMBL4610306,Inhibition of recombinant human biotinylated V...,B,None,None,BAO_0000190,BAO_0000219,cell-based format,CCc1nnc(NC(=O)CS

## Preprocesamiento de datos de bioactividad

### Combinar 3 columnas (molecule_chembl_id,canonical_smiles,standard_value) y bioactivity_class en un DataFrame

In [133]:
selection = ['molecule_chembl_id','canonical_smiles','standard_value']
df3 = df2_unique_can_smiles[selection]
df3

,molecule_chembl_id,canonical_smiles,standard_value
0,CHEMBL212560,CN(C)CCCOc1cccc2c(C#N)c(-c3ccccc3)c(NC3CCCCC3)n12,79000.0
1,CHEMBL386641,CN(C)CCNc1cccc2c(C#N)c(-c3ccccc3)c(NC3CCCCC3)n12,42000.0
2,CHEMBL425440,N#Cc1c(-c2ccccc2)c(Nc2cccc(N)c2)n2c(Cl)cccc12,80000.0
3,CHEMBL436932,CN(C)CCCNc1c(-c2ccccc2)c(C#N)c2cccc(Cl)n12,62000.0
4,CHEMBL213321,Cc1cccc(C)c1Nc1c(-c2ccccc2)c(C#N)c2cccc(Cl)n12,60000.0
...,...,...,...
112,CHEMBL4515173,C[C@H](NC(=O)[C@@H]1Cc2ccccc2CN1C(=O)[C@@H](N)...,50100.0
135,CHEMBL4646334,CCc1nnc(NC(=O)CSc2nnc(-c3ccccc3)n2-c2cccc(C)c2)s1,41100.0
136,CHEMBL4643884,CCc1nnc(NC(=O)CSc2nnc(-c3ccc(C)cc3)n2-c2cccc(C...,19100.0
137,CHEMBL4637483,CCc1nnc(NC(=O)CSc2nnc(-c3ccc(O)cc3)n2-c2cccc(C...,41700.0


Guardar resultados en un archivo CSV

In [146]:
# Subir el csv a la carpeta destino
df3.to_csv('/content/drive/My Drive/Colab Notebooks/data/bioactivity_data_preprocessed.csv', index=False)

In [147]:
# Verificar cambios
! ls '/content/drive/My Drive/Colab Notebooks/data/'

bioactivity_data.csv  bioactivity_data_preprocessed.csv


### Etiquetado de componentes
Los datos de bioactividad estan en la unidad IC50. Compuestos con menos de 1000 nM será considerados como **activos** mientras aquellos que sean mayores a 10,000 nM serán considerados como **inactivos**. Todos los valores que caen dentro de 1,000 y 10,000 nM serán clasificados como **intermedios**.

In [148]:
# Leer csv con datos preprocesados
df4 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/bioactivity_data_preprocessed.csv')

In [150]:
bioactivity_threshold = []
for i in df4.standard_value:
  if float(i) >= 10000:
    bioactivity_threshold.append("inactive")
  elif float(i) <= 1000:
    bioactivity_threshold.append("active")
  else:
    bioactivity_threshold.append("intermediate")

In [151]:
# Concatenar lista generada como una serie de pandas y agregarla al df
bioactivity_class = pd.Series(bioactivity_threshold, name='class')
df5 = pd.concat([df4, bioactivity_class], axis=1)
df5

,molecule_chembl_id,canonical_smiles,standard_value,class
0,CHEMBL212560,CN(C)CCCOc1cccc2c(C#N)c(-c3ccccc3)c(NC3CCCCC3)n12,79000.0,inactive
1,CHEMBL386641,CN(C)CCNc1cccc2c(C#N)c(-c3ccccc3)c(NC3CCCCC3)n12,42000.0,inactive
2,CHEMBL425440,N#Cc1c(-c2ccccc2)c(Nc2cccc(N)c2)n2c(Cl)cccc12,80000.0,inactive
3,CHEMBL436932,CN(C)CCCNc1c(-c2ccccc2)c(C#N)c2cccc(Cl)n12,62000.0,inactive
4,CHEMBL213321,Cc1cccc(C)c1Nc1c(-c2ccccc2)c(C#N)c2cccc(Cl)n12,60000.0,inactive
...,...,...,...,...
93,CHEMBL4515173,C[C@H](NC(=O)[C@@H]1Cc2ccccc2CN1C(=O)[C@@H](N)...,50100.0,inactive
94,CHEMBL4646334,CCc1nnc(NC(=O)CSc2nnc(-c3ccccc3)n2-c2cccc(C)c2)s1,41100.0,inactive
95,CHEMBL4643884,CCc1nnc(NC(=O)CSc2nnc(-c3ccc(C)cc3)n2-c2cccc(C...,19100.0,inactive
96,CHEMBL4637483,CCc1nnc(NC(=O)CSc2nnc(-c3ccc(O)cc3)n2-c2cccc(C...,41700.0,inactive


Guardad dataframe como archivo csv

In [152]:
# Subir el csv a la carpeta destino
df5.to_csv('/content/drive/My Drive/Colab Notebooks/data/bioactivity_data_curated.csv', index=False)

In [159]:
# Guardar archivos en un archivo zip
!zip -r /content/drive/My\ Drive/Colab\ Notebooks/data/vegf165.zip /content/drive/My\ Drive/Colab\ Notebooks/data/*.csv

updating: content/drive/My Drive/Colab Notebooks/data/bioactivity_data.csv (deflated 90%)
updating: content/drive/My Drive/Colab Notebooks/data/bioactivity_data_curated.csv (deflated 83%)
updating: content/drive/My Drive/Colab Notebooks/data/bioactivity_data_preprocessed.csv (deflated 82%)
